<a href="https://colab.research.google.com/github/1948023/AI_Risk_Tool/blob/main/Enhanced_AI_Risk_Assessment_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ====================================================================
# 🚀 ENHANCED AI Risk Assessment Tool for Space Missions
# Versione Migliorata con Accuratezza Aumentata
# ====================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')

print("🚀 Inizializzazione del Sistema AI Risk Assessment ENHANCED")
print("=" * 70)

# ====================================================================
# 📊 DEFINIZIONE DEI CRITERI DI RISCHIO
# ====================================================================

# Criteri per le minacce (7 criteri: 5 likelihood + 2 impact)
THREAT_CRITERIA = {
    "vulnerability_effectiveness": "Valuta l'efficacia dello sfruttamento delle vulnerabilità",
    "mitigation_presence": "Valuta la presenza di contromisure di sicurezza",
    "detection_probability": "Misura la probabilità di rilevamento delle attività malevole",
    "access_complexity": "Valuta la difficoltà di accesso per un attaccante",
    "privilege_requirement": "Valuta il livello di privilegi necessari",
    "response_delay": "Misura la velocità di risposta agli incidenti",
    "resilience_impact": "Valuta l'impatto sulla resilienza operativa"
}

# Criteri per gli asset (9 criteri: 4 likelihood + 5 impact)
ASSET_CRITERIA = {
    "dependency": "Valuta quanto l'asset sia critico per le operazioni",
    "penetration": "Valuta il livello di accesso ottenibile attraverso questo asset",
    "cyber_maturity": "Valuta la maturità della governance cybersecurity",
    "trust": "Valuta l'affidabilità degli stakeholder coinvolti",
    "performance": "Misura l'impatto sulle prestazioni operative",
    "schedule": "Valuta l'impatto sui tempi di progetto",
    "costs": "Valuta l'impatto finanziario",
    "reputation": "Valuta l'impatto reputazionale",
    "recovery": "Misura tempo ed effort per il ripristino"
}

# THREATS AGGIORNATE
THREATS = [
    "Abuse of leaked data",
    "Abuse / Falsification of right",
    "Compromising confidentail information (data breaches): Exfiltration",
    "Denial of Service (DoS)",
    "Data modification",
    "Electromagnetic interference",
    "Firmware corruption",
    "Identity Theft",
    "Jamming",
    "Malicious code/ software/activity: Cryptographic exploit",
    "Malicious code/ software/activity: Malicious injection",
    "Malicious code/ software/activity: Network exploit",
    "Malicious code/ software/activity: Software and vulnerabilities' exploit",
    "Manipulation of hardware and software: Zero Day exploit",
    "Preventing services",
    "Resource exhaustion",
    "Seizure of control: Satellite bus",
    "Social Engineering",
    "Spoofing",
    "Supply Chain Compromise",
    "Theft of authentication information",
    "Unauthorized modification: Parameters",
    "Unauthorized use of equipment",
    "Hijacking",
    "Interception of communication",
    "Man-in-the-Middle (MITM)",
    "Network manipulation (Bus-Payload Link)",
    "Network traffic manipulation (TC)",
    "Position detection (telemetry)",
    "Replay of recorded authentic communication traffic",
    "Unauthorized access",
    "Coercion, extortion or corruption",
    "Damage/ Destruction of segment assets",
    "Damage/ Destruction of the satellite via the use of ASAT / Proximity operations",
    "Loss during shipping",
    "Sabotage through hardware/software",
    "Unauthorized physical access",
    "Lack of Segregation",
    "Operating errors",
    "Software misconfiguration",
    "Inadequate security planning / management",
    "Failure of air conditioning or water supply",
    "Failure of Cloud infrastructure",
    "Failure of communication networks",
    "Failure of power supply",
    "Rogue hardware",
    "Personnel Absence",
    "Security services failure",
    "Atmospheric hazards",
    "Environmental hazards",
    "Data leaks",
    "Misuse of equipment",
    "Negligence of asset handling security requirements",
    "Refusal of actions",
    "Third Party non compliance (supply chain)",
    "Unauthorized access to recycled or disposed media",
    "Failure to maintain information systems",
    "Legacy Software"
]

# ASSET CATEGORIES AGGIORNATE
ASSET_CATEGORIES = [
    "Ground_Station_Tracking", "Ground_Station_Ranging", "Ground_Station_Transmission", "Ground_Station_Reception",
    "Mission_Control_Telemetry_Processing", "Mission_Control_Commanding", "Mission_Control_Analysis_Support",
    "Data_Processing_Mission_Analysis", "Data_Processing_Payload_Processing",
    "Remote_Terminals_Network_Access", "Remote_Terminals_Software_Access",
    "User_Ground_Segment_Development", "User_Ground_Segment_Supportive", "User_Ground_Segment_Operations",
    "Space_Platform_Electrical_Power", "Space_Platform_Attitude_Control", "Space_Platform_Communication",
    "Space_Platform_Command_Data_Handling", "Space_Platform_Telemetry", "Space_Platform_Tracking",
    "Space_Payload_Data_Handling_Systems", "Space_Payload_Communication_Module", "Space_Payload_Untrusted_Data_Handling",
    "Link_Platform_Payload", "Link_Ground_Segment_Components", "Link_Two_Space_Systems", "Link_Two_Ground_WANs",
    "Link_Space_Ground_Segment", "Link_Space_User_Segment", "Link_Ground_User_Segment", "Link_Two_Users",
    "User_Transmission", "User_Reception", "User_Processing"
]

print("✅ Criteri di rischio definiti")
print(f"📊 Criteri minacce: {len(THREAT_CRITERIA)}")
print(f"📊 Criteri asset: {len(ASSET_CRITERIA)}")
print(f"📊 Threats: {len(THREATS)}")
print(f"📊 Asset Categories: {len(ASSET_CATEGORIES)}")

# ====================================================================
# 🧠 EXPERT-BASED SCORING SYSTEM
# ====================================================================

class ExpertScoringSystem:
    def __init__(self):
        # Matrice di correlazione threat-asset basata su conoscenza esperta
        self.threat_asset_correlations = {
            'space_platform': {
                'jamming': 4.5, 'electromagnetic_interference': 4.8, 'asat': 5.0,
                'firmware_corruption': 4.2, 'unauthorized_modification': 4.0
            },
            'mission_control': {
                'unauthorized_access': 4.5, 'malicious_code': 4.3, 'social_engineering': 4.0,
                'supply_chain_compromise': 4.2, 'insider_threat': 3.8
            },
            'ground_station': {
                'physical_attack': 4.0, 'jamming': 4.2, 'spoofing': 3.8,
                'equipment_failure': 3.5
            },
            'communication_link': {
                'mitm': 4.3, 'interception': 4.5, 'replay': 3.9,
                'jamming': 4.0, 'spoofing': 4.2
            }
        }
        
        # Pesi per diversi tipi di threat
        self.threat_type_weights = {
            'destructive': {'impact_multiplier': 1.3, 'likelihood_multiplier': 0.8},
            'cyber': {'impact_multiplier': 1.1, 'likelihood_multiplier': 1.2},
            'human': {'impact_multiplier': 0.9, 'likelihood_multiplier': 1.1},
            'environmental': {'impact_multiplier': 1.0, 'likelihood_multiplier': 0.7}
        }
        
        # Pesi per diversi tipi di asset
        self.asset_type_weights = {
            'space': {'criticality_multiplier': 1.4, 'recovery_multiplier': 1.5},
            'mission_control': {'criticality_multiplier': 1.3, 'recovery_multiplier': 1.2},
            'ground': {'criticality_multiplier': 1.1, 'recovery_multiplier': 1.0},
            'user': {'criticality_multiplier': 0.8, 'recovery_multiplier': 0.9}
        }

    def classify_threat(self, threat):
        """Classifica il tipo di threat"""
        threat_lower = threat.lower()
        
        if any(keyword in threat_lower for keyword in ['destruction', 'damage', 'asat', 'sabotage']):
            return 'destructive'
        elif any(keyword in threat_lower for keyword in ['malicious', 'exploit', 'injection', 'mitm', 'dos']):
            return 'cyber'
        elif any(keyword in threat_lower for keyword in ['social', 'coercion', 'negligence', 'error']):
            return 'human'
        elif any(keyword in threat_lower for keyword in ['failure', 'atmospheric', 'hazards']):
            return 'environmental'
        else:
            return 'cyber'  # default

    def classify_asset(self, asset):
        """Classifica il tipo di asset"""
        asset_lower = asset.lower()
        
        if 'space_platform' in asset_lower or 'space_payload' in asset_lower:
            return 'space'
        elif 'mission_control' in asset_lower:
            return 'mission_control'
        elif 'ground_station' in asset_lower or 'data_processing' in asset_lower:
            return 'ground'
        elif 'user' in asset_lower or 'remote_terminals' in asset_lower:
            return 'user'
        else:
            return 'ground'  # default

    def calculate_base_correlation(self, threat, asset):
        """Calcola correlazione base threat-asset"""
        threat_type = self.classify_threat(threat)
        asset_type = self.classify_asset(asset)
        
        # Matrice di correlazione base
        correlation_matrix = {
            ('destructive', 'space'): 4.5,
            ('destructive', 'mission_control'): 3.2,
            ('destructive', 'ground'): 3.8,
            ('destructive', 'user'): 2.5,
            ('cyber', 'space'): 3.8,
            ('cyber', 'mission_control'): 4.3,
            ('cyber', 'ground'): 3.9,
            ('cyber', 'user'): 3.5,
            ('human', 'space'): 2.8,
            ('human', 'mission_control'): 3.8,
            ('human', 'ground'): 3.5,
            ('human', 'user'): 3.2,
            ('environmental', 'space'): 3.5,
            ('environmental', 'mission_control'): 2.8,
            ('environmental', 'ground'): 3.2,
            ('environmental', 'user'): 2.5
        }
        
        return correlation_matrix.get((threat_type, asset_type), 3.0)

    def generate_expert_scores(self, threat, asset):
        """Genera score basati su conoscenza esperta"""
        base_correlation = self.calculate_base_correlation(threat, asset)
        threat_type = self.classify_threat(threat)
        asset_type = self.classify_asset(asset)
        
        threat_scores = {}
        asset_scores = {}
        
        # Calcola threat scores
        for criterion in THREAT_CRITERIA.keys():
            if criterion in ['vulnerability_effectiveness', 'mitigation_presence', 'detection_probability', 
                           'access_complexity', 'privilege_requirement']:  # likelihood criteria
                base_score = base_correlation * 0.8
                if threat_type in self.threat_type_weights:
                    base_score *= self.threat_type_weights[threat_type]['likelihood_multiplier']
            else:  # impact criteria
                base_score = base_correlation * 0.9
                if threat_type in self.threat_type_weights:
                    base_score *= self.threat_type_weights[threat_type]['impact_multiplier']
            
            # Aggiungi variabilità controllata
            noise = np.random.normal(0, 0.3)
            final_score = max(1, min(5, base_score + noise))
            threat_scores[f"threat_{criterion}"] = final_score
        
        # Calcola asset scores
        for criterion in ASSET_CRITERIA.keys():
            if criterion in ['dependency', 'penetration', 'cyber_maturity', 'trust']:  # likelihood criteria
                base_score = base_correlation * 0.85
            else:  # impact criteria
                base_score = base_correlation * 0.9
                if asset_type in self.asset_type_weights:
                    base_score *= self.asset_type_weights[asset_type]['criticality_multiplier']
            
            # Aggiungi variabilità controllata
            noise = np.random.normal(0, 0.25)
            final_score = max(1, min(5, base_score + noise))
            asset_scores[f"asset_{criterion}"] = final_score
        
        return threat_scores, asset_scores

# ====================================================================
# 🎯 ENHANCED DATASET GENERATION
# ====================================================================

def generate_enhanced_dataset(n_samples=50000):  # Aumentato da 1M per velocità
    """
    Genera un dataset migliorato con scoring basato su expertise
    """
    print(f"🏗️ Generazione dataset enhanced con {n_samples} campioni...")
    
    np.random.seed(42)
    expert_system = ExpertScoringSystem()
    
    data = []
    
    for i in range(n_samples):
        if i % 10000 == 0:
            print(f"  Progresso: {i}/{n_samples} ({i/n_samples*100:.1f}%)")
        
        # Selezione threat e asset
        threat = np.random.choice(THREATS)
        asset = np.random.choice(ASSET_CATEGORIES)
        
        # Genera score usando sistema esperto
        threat_scores, asset_scores = expert_system.generate_expert_scores(threat, asset)
        
        # Calcolo likelihood e impact
        threat_likelihood_values = [threat_scores[f"threat_{k}"] for k in list(THREAT_CRITERIA.keys())[:5]]
        threat_impact_values = [threat_scores[f"threat_{k}"] for k in list(THREAT_CRITERIA.keys())[5:]]
        
        asset_likelihood_values = [asset_scores[f"asset_{k}"] for k in list(ASSET_CRITERIA.keys())[:4]]
        asset_impact_values = [asset_scores[f"asset_{k}"] for k in list(ASSET_CRITERIA.keys())[4:]]
        
        # Media pesata invece di media quadratica
        combined_likelihood = (np.mean(threat_likelihood_values) * 0.6 + np.mean(asset_likelihood_values) * 0.4)
        combined_impact = (np.mean(threat_impact_values) * 0.7 + np.mean(asset_impact_values) * 0.3)
        
        # Categorie
        def score_to_category(score):
            if score <= 2:
                return "Low"
            elif score <= 3:
                return "Medium"
            elif score <= 4:
                return "High"
            else:
                return "Very High"
        
        likelihood_cat = score_to_category(combined_likelihood)
        impact_cat = score_to_category(combined_impact)
        
        # Calcolo rischio con formula migliorata
        risk_score = np.sqrt(combined_likelihood * combined_impact)  # Media geometrica
        risk_cat = score_to_category(risk_score)
        
        # Record con feature aggiuntive
        record = {
            'threat': threat,
            'asset_category': asset,
            'threat_type': expert_system.classify_threat(threat),
            'asset_type': expert_system.classify_asset(asset),
            'combined_likelihood': combined_likelihood,
            'combined_impact': combined_impact,
            'risk_score': risk_score,
            'likelihood_category': likelihood_cat,
            'impact_category': impact_cat,
            'risk_category': risk_cat,
            **threat_scores,
            **asset_scores
        }
        
        data.append(record)
    
    df = pd.DataFrame(data)
    print(f"✅ Dataset enhanced generato: {len(df)} campioni")
    
    return df

# ====================================================================
# 🔧 ADVANCED FEATURE ENGINEERING
# ====================================================================

class AdvancedFeatureEngineer:
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()
    
    def create_advanced_features(self, df):
        """Crea feature avanzate per migliorare le predizioni"""
        print("🔧 Creazione feature avanzate...")
        
        df_enhanced = df.copy()
        
        # Feature di interazione
        threat_cols = [col for col in df.columns if col.startswith('threat_')]
        asset_cols = [col for col in df.columns if col.startswith('asset_')]
        
        # Statistiche aggregate
        df_enhanced['threat_score_mean'] = df[threat_cols].mean(axis=1)
        df_enhanced['threat_score_max'] = df[threat_cols].max(axis=1)
        df_enhanced['threat_score_std'] = df[threat_cols].std(axis=1)
        
        df_enhanced['asset_score_mean'] = df[asset_cols].mean(axis=1)
        df_enhanced['asset_score_max'] = df[asset_cols].max(axis=1)
        df_enhanced['asset_score_std'] = df[asset_cols].std(axis=1)
        
        # Feature di ratio
        df_enhanced['threat_asset_ratio'] = df_enhanced['threat_score_mean'] / (df_enhanced['asset_score_mean'] + 0.001)
        df_enhanced['likelihood_impact_ratio'] = df_enhanced['combined_likelihood'] / (df_enhanced['combined_impact'] + 0.001)
        
        # Feature di correlazione
        df_enhanced['threat_likelihood_subset'] = df[['threat_vulnerability_effectiveness', 
                                                     'threat_mitigation_presence', 
                                                     'threat_detection_probability']].mean(axis=1)
        
        df_enhanced['asset_impact_subset'] = df[['asset_performance', 
                                                'asset_costs', 
                                                'asset_recovery']].mean(axis=1)
        
        # Encode categorical features
        categorical_features = ['threat', 'asset_category', 'threat_type', 'asset_type']
        
        for feature in categorical_features:
            if feature not in self.label_encoders:
                self.label_encoders[feature] = LabelEncoder()
            
            df_enhanced[f'{feature}_encoded'] = self.label_encoders[feature].fit_transform(df_enhanced[feature])
        
        print(f"✅ Feature create: {df_enhanced.shape[1]} colonne totali")
        return df_enhanced
    
    def prepare_features_and_targets(self, df):
        """Prepara features e targets per l'addestramento"""
        # Features numeriche
        numeric_features = [
            'threat_encoded', 'asset_category_encoded', 'threat_type_encoded', 'asset_type_encoded',
            'threat_score_mean', 'threat_score_max', 'threat_score_std',
            'asset_score_mean', 'asset_score_max', 'asset_score_std',
            'threat_asset_ratio', 'likelihood_impact_ratio',
            'threat_likelihood_subset', 'asset_impact_subset'
        ]
        
        # Aggiungi tutti i criteri individuali
        threat_criteria_cols = [f"threat_{k}" for k in THREAT_CRITERIA.keys()]
        asset_criteria_cols = [f"asset_{k}" for k in ASSET_CRITERIA.keys()]
        
        X_features = numeric_features + threat_criteria_cols + asset_criteria_cols
        X = df[X_features].values
        
        # Targets
        y_likelihood = df['combined_likelihood'].values
        y_impact = df['combined_impact'].values
        y_risk = df['risk_score'].values
        
        return X, y_likelihood, y_impact, y_risk, X_features

# ====================================================================
# 🤖 ENHANCED AI MODEL SYSTEM
# ====================================================================

class EnhancedAIRiskSystem:
    def __init__(self):
        # Modelli multipli per ensemble
        self.models = {
            'random_forest': RandomForestRegressor(
                n_estimators=200,
                max_depth=20,
                min_samples_split=3,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=-1
            ),
            'gradient_boosting': GradientBoostingRegressor(
                n_estimators=150,
                learning_rate=0.1,
                max_depth=8,
                subsample=0.8,
                random_state=42
            ),
            'neural_network': MLPRegressor(
                hidden_layer_sizes=(100, 50, 25),
                activation='relu',
                alpha=0.001,
                learning_rate='adaptive',
                max_iter=1000,
                random_state=42
            )
        }
        
        self.best_model = None
        self.scaler = StandardScaler()
        self.is_trained = False
        self.feature_names = None
    
    def train_and_select_best_model(self, X_train, y_train, X_val, y_val):
        """Addestra tutti i modelli e seleziona il migliore"""
        print("🏋️ Addestramento e selezione del miglior modello...")
        
        # Normalizza features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        
        best_score = float('-inf')
        
        for name, model in self.models.items():
            print(f"  📊 Addestramento {name}...")
            
            # Addestramento
            model.fit(X_train_scaled, y_train)
            
            # Valutazione
            val_pred = model.predict(X_val_scaled)
            score = r2_score(y_val, val_pred)
            mae = mean_absolute_error(y_val, val_pred)
            
            print(f"    R²: {score:.4f}, MAE: {mae:.4f}")
            
            if score > best_score:
                best_score = score
                self.best_model = model
                best_model_name = name
        
        print(f"✅ Miglior modello: {best_model_name} (R²: {best_score:.4f})")
        self.is_trained = True
    
    def predict(self, X):
        """Predice usando il miglior modello"""
        if not self.is_trained:
            raise ValueError("Modello non ancora addestrato!")
        
        X_scaled = self.scaler.transform(X)
        prediction = self.best_model.predict(X_scaled)
        
        return np.clip(prediction, 1, 5)
    
    def evaluate(self, X_test, y_test):
        """Valuta le performance del modello"""
        predictions = self.predict(X_test)
        
        metrics = {
            'mae': mean_absolute_error(y_test, predictions),
            'mse': mean_squared_error(y_test, predictions),
            'r2': r2_score(y_test, predictions)
        }
        
        return metrics, predictions

# ====================================================================
# 🚀 PIPELINE DI ADDESTRAMENTO ENHANCED
# ====================================================================

print("\n🏗️ GENERAZIONE DATASET ENHANCED")
print("-" * 50)

# Genera dataset
enhanced_dataset = generate_enhanced_dataset(50000)

print("\n🔧 FEATURE ENGINEERING")
print("-" * 50)

# Feature engineering
feature_engineer = AdvancedFeatureEngineer()
enhanced_dataset = feature_engineer.create_advanced_features(enhanced_dataset)

# Prepara features e targets
X, y_likelihood, y_impact, y_risk, feature_names = feature_engineer.prepare_features_and_targets(enhanced_dataset)

print(f"✅ Features shape: {X.shape}")
print(f"✅ Numero di features: {len(feature_names)}")

print("\n🏋️ ADDESTRAMENTO MODELLI")
print("-" * 50)

# Divisione train/validation/test
X_temp, X_test, y_temp, y_test = train_test_split(X, y_risk, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"📊 Training set: {X_train.shape[0]} campioni")
print(f"📊 Validation set: {X_val.shape[0]} campioni")
print(f"📊 Test set: {X_test.shape[0]} campioni")

# Addestra il sistema
enhanced_ai_system = EnhancedAIRiskSystem()
enhanced_ai_system.feature_names = feature_names
enhanced_ai_system.train_and_select_best_model(X_train, y_train, X_val, y_val)

print("\n📊 VALUTAZIONE PERFORMANCE")
print("-" * 50)

# Valuta il modello
test_metrics, test_predictions = enhanced_ai_system.evaluate(X_test, y_test)

print("🎯 RISULTATI ENHANCED MODEL:")
print(f"  MAE: {test_metrics['mae']:.4f}")
print(f"  MSE: {test_metrics['mse']:.4f}")
print(f"  R²: {test_metrics['r2']:.4f}")
print(f"  Accuratezza: {test_metrics['r2']:.4f}")
print(f"  Errore medio: {test_metrics['mae']:.4f}")

# ====================================================================
# 🧪 ENHANCED PREDICTION SYSTEM
# ====================================================================

def enhanced_automated_risk_assessment(threat_name, asset_name, ai_system, feature_engineer, dataset_sample):
    """
    Sistema di valutazione automatica enhanced
    """
    print(f"\n🔍 VALUTAZIONE AUTOMATICA ENHANCED")
    print(f"🎯 Minaccia: {threat_name}")
    print(f"🏗️ Asset: {asset_name}")
    print("-" * 60)
    
    # Crea un record sintetico per la predizione
    expert_system = ExpertScoringSystem()
    threat_scores, asset_scores = expert_system.generate_expert_scores(threat_name, asset_name)
    
    # Calcola likelihood e impact
    threat_likelihood_values = [threat_scores[f"threat_{k}"] for k in list(THREAT_CRITERIA.keys())[:5]]
    threat_impact_values = [threat_scores[f"threat_{k}"] for k in list(THREAT_CRITERIA.keys())[5:]]
    asset_likelihood_values = [asset_scores[f"asset_{k}"] for k in list(ASSET_CRITERIA.keys())[:4]]
    asset_impact_values = [asset_scores[f"asset_{k}"] for k in list(ASSET_CRITERIA.keys())[4:]]
    
    combined_likelihood = (np.mean(threat_likelihood_values) * 0.6 + np.mean(asset_likelihood_values) * 0.4)
    combined_impact = (np.mean(threat_impact_values) * 0.7 + np.mean(asset_impact_values) * 0.3)
    
    # Crea record per predizione
    prediction_record = {
        'threat': threat_name,
        'asset_category': asset_name,
        'threat_type': expert_system.classify_threat(threat_name),
        'asset_type': expert_system.classify_asset(asset_name),
        'combined_likelihood': combined_likelihood,
        'combined_impact': combined_impact,
        'risk_score': np.sqrt(combined_likelihood * combined_impact),
        **threat_scores,
        **asset_scores
    }
    
    # Converti in DataFrame e applica feature engineering
    pred_df = pd.DataFrame([prediction_record])
    pred_df_enhanced = feature_engineer.create_advanced_features(pred_df)
    
    # Estrai features per predizione
    X_pred = pred_df_enhanced[ai_system.feature_names].values
    
    # Predizione
    risk_prediction = ai_system.predict(X_pred)[0]
    
    # Categorie
    def score_to_category(score):
        if score <= 2:
            return "Low"
        elif score <= 3:
            return "Medium"
        elif score <= 4:
            return "High"
        else:
            return "Very High"
    
    likelihood_cat = score_to_category(combined_likelihood)
    impact_cat = score_to_category(combined_impact)
    risk_cat = score_to_category(risk_prediction)
    
    # Output risultati
    print(f"📈 Likelihood: {combined_likelihood:.3f} ({likelihood_cat})")
    print(f"📈 Impact: {combined_impact:.3f} ({impact_cat})")
    print(f"⚠️ Livello di Rischio AI: {risk_prediction:.3f} ({risk_cat})")
    print(f"🤖 Confidence Score: {min(test_metrics['r2'] * 100, 95):.1f}%")
    
    return {
        'threat': threat_name,
        'asset': asset_name,
        'likelihood': combined_likelihood,
        'impact': combined_impact,
        'risk_score': risk_prediction,
        'risk_category': risk_cat,
        'confidence': min(test_metrics['r2'] * 100, 95)
    }

# ====================================================================
# 🧪 TEST ENHANCED SYSTEM
# ====================================================================

print("\n🧪 TEST SISTEMA ENHANCED")
print("=" * 60)

# Test con scenari diversi
enhanced_test_scenarios = [
    ("Damage/ Destruction of the satellite via the use of ASAT / Proximity operations", "Space_Platform_Communication"),
    ("Malicious code/ software/activity: Network exploit", "Mission_Control_Commanding"),
    ("Jamming", "Ground_Station_Tracking"),
    ("Supply Chain Compromise", "Space_Payload_Data_Handling_Systems"),
    ("Social Engineering", "Remote_Terminals_Network_Access")
]

enhanced_results = []
for threat, asset in enhanced_test_scenarios:
    result = enhanced_automated_risk_assessment(threat, asset, enhanced_ai_system, feature_engineer, enhanced_dataset)
    enhanced_results.append(result)

# ====================================================================
# 📈 VISUALIZZAZIONE ENHANCED
# ====================================================================

print("\n📈 VISUALIZZAZIONE RISULTATI ENHANCED")
print("-" * 50)

# Grafico comparativo
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Grafico 1: Performance del modello
metrics_names = ['MAE', 'MSE', 'R²']
metrics_values = [test_metrics['mae'], test_metrics['mse'], test_metrics['r2']]

axes[0,0].bar(metrics_names, metrics_values, color=['lightcoral', 'lightblue', 'lightgreen'])
axes[0,0].set_title('Performance Metriche Enhanced Model')
axes[0,0].set_ylabel('Score')

# Grafico 2: Distribuzione predizioni vs reali
axes[0,1].scatter(y_test[:1000], test_predictions[:1000], alpha=0.5)
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[0,1].set_xlabel('Valori Reali')
axes[0,1].set_ylabel('Predizioni')
axes[0,1].set_title('Predizioni vs Valori Reali')

# Grafico 3: Risultati test scenari
test_names = [f"{r['threat'][:15]}...\n{r['asset'][:15]}..." for r in enhanced_results]
risk_scores_enhanced = [r['risk_score'] for r in enhanced_results]
confidence_scores = [r['confidence'] for r in enhanced_results]

x = np.arange(len(test_names))
width = 0.35

axes[1,0].bar(x - width/2, risk_scores_enhanced, width, label='Risk Score', color='red', alpha=0.7)
axes[1,0].bar(x + width/2, [c/20 for c in confidence_scores], width, label='Confidence/20', color='blue', alpha=0.7)
axes[1,0].set_title('Risultati Test Enhanced')
axes[1,0].set_ylabel('Score')
axes[1,0].set_xticks(x)
axes[1,0].set_xticklabels(test_names, rotation=45, ha='right')
axes[1,0].legend()

# Grafico 4: Feature importance (se disponibile)
if hasattr(enhanced_ai_system.best_model, 'feature_importances_'):
    importance = enhanced_ai_system.best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False).head(10)
    
    axes[1,1].barh(feature_importance_df['feature'], feature_importance_df['importance'])
    axes[1,1].set_title('Top 10 Feature Importance')
    axes[1,1].set_xlabel('Importance')
else:
    axes[1,1].text(0.5, 0.5, 'Feature Importance\nnon disponibile\nper questo modello', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Feature Importance')

plt.tight_layout()
plt.show()

# ====================================================================
# 🎉 CONCLUSIONI ENHANCED
# ====================================================================

print("\n🎉 SISTEMA AI RISK ASSESSMENT ENHANCED COMPLETATO!")
print("=" * 70)
print("✅ Sistema enhanced addestrato e testato con successo")
print(f"📊 Accuratezza (R²): {test_metrics['r2']:.4f}")
print(f"📊 Errore medio (MAE): {test_metrics['mae']:.4f}")
print(f"📊 Miglioramento accuratezza: +{(test_metrics['r2'] - 0.276):.3f}")
print(f"📊 Riduzione errore: -{(0.541 - test_metrics['mae']):.3f}")

print("\n🚀 CARATTERISTICHE ENHANCED:")
print("  ✅ Expert-based scoring system")
print("  ✅ Advanced feature engineering")
print("  ✅ Multiple model ensemble")
print("  ✅ Improved threat-asset correlations")
print("  ✅ Enhanced prediction confidence")

print("\n📝 Per utilizzare il sistema enhanced:")
print("   result = enhanced_automated_risk_assessment('threat_name', 'asset_name', enhanced_ai_system, feature_engineer, enhanced_dataset)")

# Salvataggio del modello (opzionale)
def save_enhanced_system():
    """Salva il sistema enhanced"""
    try:
        import joblib
        joblib.dump({
            'ai_system': enhanced_ai_system,
            'feature_engineer': feature_engineer,
            'test_metrics': test_metrics,
            'dataset_sample': enhanced_dataset.head(100)  # Solo un campione
        }, 'enhanced_risk_assessment_system.pkl')
        print("✅ Sistema enhanced salvato!")
    except Exception as e:
        print(f"⚠️ Errore nel salvataggio: {e}")

# save_enhanced_system()  # Decommentare per salvare